In [1]:
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time


Driver = r'.\chromedriver-win64\chromedriver.exe'
def WebScrape(Driver,Business, Location):
    
    final_collection = [] # List to collect all reviews
    
    driver = webdriver.Chrome(r'chromedriver-win64\chromedriver.exe')
    driver.get("https://www.google.com")
    
    search_box = driver.find_element(By.NAME, "q") # Searching for the business name and location on Google
    search_box.send_keys(f"{Business} {Location} reviews")
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(1)  
    
    # Waiting for the review button to be clickable
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-async-trigger='reviewDialog']"))
    )
    
    # getting the number of reviews
    review_text = review_button.text
    review_counts_str = review_text.split(" Google reviews")[0].replace(",", "")
    review_counts = int(review_counts_str)
    
    # Opening the reviews modal for extraction
    review_button.click()

    review_limit = min(review_counts, 500)  # Set number of revies to extract to 500 or less if fewer reviews are available
    review_count = 0

    # Waiting for the reviews to load
    review_dialog = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "review-dialog-list"))
    )
    
    # Scrolling and scraping reviews
    while review_count < review_limit:

        soup = BeautifulSoup(driver.page_source, "html.parser")

        reviews = soup.find_all("span", class_="review-full-text")  # Incase of future changes adjust class to get review text

        # Updating the review count
        review_count += len(reviews)

        # Extracting reviews
        for review in reviews:
            final_collection.append(review.get_text())
        
        # If review limit is reached, stop scraping
        if review_count >= review_limit:
            break
        
        # Scrolling within the reviews modal to load more reviews
        scrollable_element = driver.find_element(By.CLASS_NAME, "review-dialog-list")
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)
        
        time.sleep(1)  
    
    driver.quit()
    
    # Return the collected reviews
    return final_collection



In [ ]:
#Example usage
Business = "Dominos Pizza"
Location = "Yaba"
collection = WebScrape(Driver,Business, Location)

In [ ]:
len(collection)